In [2]:
import pandas as pd
import os
import requests

#Download data from brasil io and pass to read_csv
df = pd.read_csv('cases-brazil-cities-time.csv')


#Filter date until the one you wants
df = df.query("date <= '2020-07-31'")
df = df.loc[df['state'] != "TOTAL"]

#Create a list of capitals
capitais = ["Rio Branco", "Maceió", "Macapá", "Manaus", "Salvador", "Fortaleza", "Brasília", "Vitória",
            "Goiânia", "São Luís", "Cuiabá", "Campo Grande", "Belo Horizonte", "Belém", "João Pessoa",
            "Curitiba", "Recife", "Teresina", "Rio de Janeiro", "Natal", "Porto Alegre", "Porto Velho",
            "Boa Vista", "Florianópolis", "São Paulo", "Aracaju", "Palmas"]
states = ["AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB",
          "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]

#Create state data
def createSeries(data, state, columns):
    data = data[data["state"] == state]
    data = data.groupby(["date", "state"])[columns].sum().reset_index()
    data = data.rename(columns = {"totalCases": "cases",
                                  "deaths": "deaths"})
    return(data)

for state in states:
    tempdf = createSeries(data = df, state = state, columns = ["totalCases", "deaths"])
    
    #Create directory
    if os.path.exists(state):
        tempdf.to_csv(state + "/" + "estado" + ".csv", index = False)
    
    else:
        os.mkdir(state)
        tempdf.to_csv(state + "/" + "estado" + ".csv", index = False)
        
#Create capital series
for state, capital in zip(states,capitais):
    tempdf = df[df["state"] == state]
    tempdf = tempdf[tempdf["city"] == capital+'/'+state]
    tempdf = tempdf[["date", "state", "city", "totalCases", "deaths"]]
    tempdf = tempdf.rename(columns = {"totalCases": "cases",
                                      "deaths": "deaths"})
    tempdf["date"] = pd.to_datetime(tempdf["date"])
    tempdf = tempdf.sort_values(by = "date")
    
    tempdf.to_csv(state + "/" + "capital" + ".csv", index = False)
    
#create series for interior
for state, capital in zip(states,capitais):
    tempdf = df[df["state"] == state]
    tempdf = tempdf[tempdf["city"] != capital+'/'+state]
    tempdf = createSeries(data = tempdf, state = state, columns = ["totalCases", "deaths"])
    tempdf.to_csv(state + "/" + "interior" + ".csv", index = False)
    
